## Importing Libraries

In [98]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import warnings
warnings.filterwarnings("ignore")

## Retrieving data from Sitemap

In [2]:
base_url = 'https://www.ghanaweb.com'

In [3]:
URL = "https://www.ghanaweb.com/GhanaHomePage/sitemap.php"
page = requests.get(URL)

#print(page.text)

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
results = soup.find(id = "medsection2")

In [6]:
site_map_data = results.find_all("dl", class_ = "resources")

In [7]:
df_columns = ['Section','url']
sections_df = pd.DataFrame(columns = df_columns)

In [8]:
for sections in site_map_data:
    urls = sections.find_all("a")
    for url in urls:
        temp_row = pd.DataFrame(data = [[url.text,base_url+url.get('href')]], columns = df_columns)
        sections_df = pd.concat([sections_df,temp_row], ignore_index=True)

In [91]:
sections_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Sections_data.csv")

## Selecting Story Sections with articles

In [46]:
section_list = ["Abroad, Ghanaians","AFCON","African News","Athletics","BBC Hausa News", "BBC Pidgin News","Business & Economy",
"Coronavirus","Crime News","Editorial News","Entertainers","Entertainment","Health News","Lifestyle","Music","Political News",
"Regional News","Sports Section","Tabloid News"]

In [79]:
def get_section_urls(row):
    df = pd.DataFrame()
    if row['Section'] in section_list:
        temp_row = pd.DataFrame(data = [[row["Section"], row["url"]]], columns = df_columns)
        df = pd.concat([df,temp_row],ignore_index =True)
    else:
        pass

    return df

In [80]:
df = sections_df.apply(lambda row : get_section_urls(row),axis = 1)

In [84]:
df_columns = ['Section Title','Url']
stories_urls_df = pd.DataFrame(columns = df_columns)

In [85]:
for row in df:
    stories_urls_df = stories_urls_df.append(row)

In [89]:
stories_urls_df.reset_index(inplace=True)

In [90]:
del stories_urls_df['index']
stories_urls_df.head()

,Section Title,Url
0,"Abroad, Ghanaians",https://www.ghanaweb.com/GhanaHomePage/diaspora/
1,AFCON,https://www.ghanaweb.com/afcon/
2,African News,https://www.ghanaweb.com/GhanaHomePage/africa/
3,Athletics,https://www.ghanaweb.com/GhanaHomePage/athletics/
4,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hausa


In [92]:
stories_urls_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Selected_Sections_data.csv")

## Getting news urls from selected sections

In [122]:
def get_story_urls(row):
    try:
        df = pd.DataFrame()
        scraper = cloudscraper.create_scraper()
        page = scraper.get(row["Url"])
        soup = BeautifulSoup(page.content, 'html.parser')
        page_stories = soup.find_all("div", class_ = "afcon-news list")
        for ul in page_stories:
            for li in ul.findAll('li'):
                data = li.find("a")
                url = row['Url']+data.get('href')
                temp_row = pd.DataFrame(data = [[row["Section Title"], url]], columns = df_columns)
                df = pd.concat([df,temp_row], ignore_index=True)  
            time.sleep(20)
    except:
        pass

    return df
    

In [123]:
url_df  = stories_urls_df.apply(lambda row : get_story_urls(row),axis = 1)

In [124]:
df_columns = ['Section Title','Url']
article_urls_df = pd.DataFrame(columns = df_columns)

In [125]:
for row in url_df:
    article_urls_df = article_urls_df.append(row)

In [128]:
article_urls_df.head()

,Section Title,Url
0,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hau...
1,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hau...
2,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hau...
3,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hau...
4,BBC Hausa News,https://www.ghanaweb.com/GhanaHomePage/bbc/hau...


In [127]:
article_urls_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Story Urls.csv")

In [129]:
## add update to include scraping data from multiple pages